In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os 
import glob
import cv2
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D

In [ ]:
pastas_img = ['train' , 'train_mask']
caminho_base = os.path.abspath('..')

def caminho_image(numero_pasta, formato):
    caminho_pastas_imgs = caminho_base + '/image/' + str(pastas_img[numero_pasta]) + '/'
    caminho_imgs = glob.glob(caminho_pastas_imgs + '*.' + str(formato))
    return caminho_imgs

In [ ]:
SIZE = 1024  #padronizar o tamanho das imagens 
train = [] #colocar as imagens em forma de matriz
mask  = []
caminho_train = caminho_image(0, 'png')
caminho_mask  = caminho_image(1, 'tiff')

for varrer in range(len(caminho_train)):
    image = plt.imread(caminho_train[varrer])
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB )
    image_resize = cv2.resize(image, (SIZE,SIZE))
    train.append(image_resize)
    
    
    

for varrer in range(len(caminho_mask)):
    image = plt.imread(caminho_mask[varrer])
    image_resize = cv2.resize(image, (SIZE,SIZE))
    mask.append(image_resize)

train = np.array(train)
mask = np.array(mask)

In [ ]:
X_train = train 
y_train = mask
y_train = np.expand_dims(y_train, axis = 3)
#importante analizar as dimensoes das imagens. X_train é uma matriz (10, 64,64,3) q significa q a mesma tem 10 imagens com largura 64x64 e tres canais de cores RGB
#logo para relizar as operações é preciso q y_train tbm tenha esse shape, por isso é necessario realizar a expand_dims


In [ ]:
feature = Sequential()
feature.add(Conv2D(filters=64, kernel_size=(3, 3), input_shape=(SIZE,SIZE, 3), strides=1, padding='same', activation='relu'))
feature.add(Conv2D(filters=64, padding='same' , kernel_size=(3, 3), strides=1, activation='relu'))
feature.add(BatchNormalization())

#tudo q fazer aqui tem q resutar no msm numero de linhas para X e Y

X = feature.predict(X_train)

In [ ]:
X = X.reshape(-1, X.shape[3])
Y = y_train.reshape(-1)

In [ ]:
df = pd.DataFrame(X)
df.head()

In [ ]:
df['Label'] = Y
df.head()

In [ ]:
df = df[df['Label'] > 0 ]

In [ ]:
X_train = df.drop(['Label'], axis=1)
y_train = df['Label']
#podemos entao agora treinar o modelo

In [ ]:
from sklearn.ensemble import RandomForestClassifier
import pickle
#model = RandomForestClassifier(n_estimators = 50, random_state = 42, n_jobs= -1)

# Train the model on training data
# Ravel Y to pass 1d array instead of column vector
#model.fit(X_train, y_train) #For sklearn no one hot encoding


#filename = 'RF_model.sav'
#pickle.dump(model, open(filename, 'wb'))

loaded_model = pickle.load(open(filename, 'rb'))



#READ EXTERNAL IMAGE...
test_img = cv2.imread('136.png', cv2.IMREAD_COLOR)       
test_img = cv2.resize(test_img, (SIZE, SIZE))
test_img = cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB)
test_img = np.expand_dims(test_img, axis=0)

#predict_image = np.expand_dims(X_train[8,:,:,:], axis=0)
X_test_feature = feature.predict(test_img)
X_test_feature = X_test_feature.reshape(-1, X_test_feature.shape[3])

prediction = loaded_model.predict(X_test_feature)

prediction_image = prediction.reshape(mask.shape[1:])
plt.imshow(prediction_image)

In [ ]:
print(asd')